In [2]:
torch.__version__
!gcc --version

gcc (Debian 8.3.0-6) 8.3.0
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html --user 
!pip install cython pyyaml==5.1 --ignore-installed
!pip install awscli
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
import torch

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9 MB 11 kB/s  eta 0:00:011
     |████████████████████████████████| 4.0 MB 2.0 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 2.0 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 274 kB 24.2 MB/s eta 0:00:01
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=e2db9e0ed9cb0f7ca3cfe7037fba4d29411dae4e6b623c4ea9ce2ebd12e475b2
  Stored in directory: /home/jupyter/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-jv0fapgd
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=326772 sha256=a79924cf3c7a29299028a87200eb632bf80af2d0bda75fa176

ModuleNotFoundError: No module named 'torch'

In [3]:
import logging
import os
import argparse
from collections import OrderedDict
import pandas as pd
import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    MetadataCatalog,
    build_detection_train_loader,
    build_detection_test_loader,
 
)
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import (
    CommonMetricPrinter,
    EventStorage,
    JSONWriter,
    TensorboardXWriter
)

from detectron2.utils.visualizer import Visualizer
from detectron2.engine import default_argument_parser, default_setup, launch
from detectron2.evaluation import ( 
    COCOEvaluator,
    DatasetEvaluators,
    inference_on_dataset,
    print_csv_format
)
 



In [4]:

import cv2
import json
import random
import os
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow


1.4.0+cu100
gcc (Debian 8.3.0-6) 8.3.0
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [5]:
subset = ["Toilet",
          "Swimming_pool",
          "Bed",
          "Billiard_table",
          "Sink",
          "Fountain",
          "Oven",
          "Ceiling_fan",
          "Television",
          "Microwave_oven",
          "Gas_stove",
          "Refrigerator",
          "Kitchen_&_dining_room_table",
          "Washing_machine",
          "Bathtub",
          "Stairs",
          "Fireplace",
          "Pillow",
          "Mirror",
          "Shower",
          "Couch",
          "Countertop",
          "Coffeemaker",
          "Dishwasher",
          "Sofa_bed",
          "Tree_house",
          "Towel",
          "Porch",
          "Wine_rack",
          "Jacuzzi"]


'Toilet,Swimming pool,Bed,Billiard table,Sink,Fountain,Oven,Ceiling fan,Television,Microwave oven,Gas stove,Refrigerator,Kitchen & dining room table,Washing machine,Bathtub,Stairs,Fireplace,Pillow,Mirror,Shower,Couch,Countertop,Coffeemaker,Dishwasher,Sofa bed,Tree house,Towel,Porch,Wine rack,Jacuzzi'

In [6]:
subset = [x.replace("_"," ") for x in subset]
%cd data 

[Errno 2] No such file or directory: 'data'
/home/jupyter/airbnb/data


In [8]:
subset

['Toilet',
 'Swimming pool',
 'Bed',
 'Billiard table',
 'Sink',
 'Fountain',
 'Oven',
 'Ceiling fan',
 'Television',
 'Microwave oven',
 'Gas stove',
 'Refrigerator',
 'Kitchen & dining room table',
 'Washing machine',
 'Bathtub',
 'Stairs',
 'Fireplace',
 'Pillow',
 'Mirror',
 'Shower',
 'Couch',
 'Countertop',
 'Coffeemaker',
 'Dishwasher',
 'Sofa bed',
 'Tree house',
 'Towel',
 'Porch',
 'Wine rack',
 'Jacuzzi']

In [10]:
#  download bouding box info data 

!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
!wget https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv


--2020-11-30 18:50:27--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.214.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1.1G) [text/csv]
Saving to: ‘train-annotations-bbox.csv’

train-annotations-b 100%[===================>]   1.11G   101MB/s    in 11s     

2020-11-30 18:50:39 (101 MB/s) - ‘train-annotations-bbox.csv’ saved [1194033454/1194033454]

--2020-11-30 18:50:39--  https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.12.128, 74.125.26.128, 172.217.193.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25105048 (24M) [

In [15]:
!python3 downloadOI.py --classes 'Toilet,Swimming pool,Bed,Billiard table,Sink,Fountain,Oven,Ceiling fan,Television,Microwave oven,Gas stove,Refrigerator,Kitchen & dining room table,Washing machine,Bathtub,Stairs,Fireplace,Pillow,Mirror,Shower,Couch,Countertop,Coffeemaker,Dishwasher,Sofa bed,Tree house,Towel,Porch,Wine rack,Jacuzzi' --dataset train
!python3 downloadOI.py --classes 'Toilet,Swimming pool,Bed,Billiard table,Sink,Fountain,Oven,Ceiling fan,Television,Microwave oven,Gas stove,Refrigerator,Kitchen & dining room table,Washing machine,Bathtub,Stairs,Fireplace,Pillow,Mirror,Shower,Couch,Countertop,Coffeemaker,Dishwasher,Sofa bed,Tree house,Towel,Porch,Wine rack,Jacuzzi' --dataset validation

CPU count: 8
grep /m/09g1w ./train-annotations-bbox.csv
grep /m/0b_rs ./train-annotations-bbox.csv
grep /m/03ssj5 ./train-annotations-bbox.csv
grep /m/04p0qw ./train-annotations-bbox.csv
grep /m/0130jx ./train-annotations-bbox.csv
grep /m/0220r2 ./train-annotations-bbox.csv
grep /m/029bxz ./train-annotations-bbox.csv
grep /m/03ldnb ./train-annotations-bbox.csv
grep /m/07c52 ./train-annotations-bbox.csv
grep /m/0fx9l ./train-annotations-bbox.csv
grep /m/02wv84t ./train-annotations-bbox.csv
grep /m/040b_t ./train-annotations-bbox.csv
grep /m/0h8n5zk ./train-annotations-bbox.csv
grep /m/0174k2 ./train-annotations-bbox.csv
grep /m/03dnzn ./train-annotations-bbox.csv
grep /m/01lynh ./train-annotations-bbox.csv
grep /m/03tw93 ./train-annotations-bbox.csv
grep /m/034c16 ./train-annotations-bbox.csv
grep /m/054_l ./train-annotations-bbox.csv
grep /m/02f9f_ ./train-annotations-bbox.csv
grep /m/02crq1 ./train-annotations-bbox.csv
grep /m/0b3fp9 ./train-annotations-bbox.csv
grep /m/07xyvk ./train

In [7]:
import pandas as pd
train_df = pd.read_csv("train-annotations-bbox.csv")
valid_df = pd.read_csv("validation-annotations-bbox.csv")

In [8]:
target=subset
target.sort()
valid_path = "./validation"
train_path = "./train"

## 自己加的
subset = target

In [9]:
def find_image(folder):

    return [os.path.splitext(x)[0] for x in os.listdir(image_folder) if x.endswith(".jpg")]

In [10]:

def format_annotations(folder, annotation_file, target):

    image_ids = find_image(folder)
    annotation_file = pd.read_csv(annotation_file)
    classes = pd.read_csv("class-descriptions-boxable.csv",
                          names=["Label", "Class"])
    annotation_file["Class"] = annotation_file["Label"].map(classes.set_index("Label")["Class"])
    annotation_file.sort_values(by=["Class"], inplace=True)
    if target is not None:
        annotation_file = annotation_file[annotation_file["ImageID"].isin(image_ids) & annotation_file["Class"].isin(target)]
    else:
        annotation_file = annotation_file[annotation_file["ImageID"].isin(image_ids)]
    annotation_file["Class"] = pd.Categorical(annotation_file["Class"])
    annotation_file["ClassID"] = annotation_file["Class"].cat.codes
    
    return annotation_file

In [11]:
def rel_to_absolute(bbox, height, width):
    bbox[0] = np.round(np.multiply(bbox[0], width)) 
    bbox[1] = np.round(np.multiply(bbox[1], height)) 
    bbox[2] = np.round(np.multiply(bbox[2], width)) 
    bbox[3] = np.round(np.multiply(bbox[3], height)) 
    return [i.astype("object") for i in bbox] 


In [12]:
import numpy as np
from detectron2.structures import BoxMode
import json
import os

def get_image_dicts(folder, annotation_file, target):
    annotations = format_annotations(folder=folder, 
                                     annotation_file=annotation_file,
                                     target=target)

    img_ids = find_image(folder)

    img_dicts = []
    for idx, img in tqdm(enumerate(img_ids)):
        record = {}
        file_name = folder + "/" + img + ".jpg"
        height, width = cv2.imread(file_name).shape[:2]
        img_data = annotations[annotations["ImageID"] == img].reset_index() 

        record["file_name"] = file_name
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        img_annotations = []
        ## write the required information in the format detectron required
        for i in range(len(img_data)): 
            category_id = img_data.loc[i]["ClassID"].astype("object") 
            bbox = np.float32(img_data.loc[i][["XMin", "YMin", "XMax", "YMax"]].values) 
            bbox = rel_to_absolute(bbox, height, width)
            annot = {
                "bbox": bbox, 
                "bbox_mode": BoxMode.XYXY_ABS,  
                "category_id": category_id
            }
            img_annotations.append(annot)
            
        record["annotations"] = img_annotations
        img_dicts.append(record)
    return img_dicts

In [25]:
### mapping raw data and transforma both train and validation into detectron2 format
%%time
valid_img_dicts = get_image_dicts(valid_path,"validation-annotations-bbox.csv",target=subset)
train_img_dicts = get_image_dicts(train_path,"train-annotations-bbox.csv",target=subset)

Using validation-annotations-bbox.csv for annotations...


5it [00:00, 48.84it/s]

On dataset: validation
Classes we're using:
 Swimming pool                  161
Bed                            125
Kitchen & dining room table     76
Pillow                          76
Countertop                      68
Couch                           61
Sofa bed                        61
Sink                            57
Porch                           52
Stairs                          45
Television                      44
Fireplace                       41
Washing machine                 40
Toilet                          37
Oven                            36
Mirror                          33
Billiard table                  32
Microwave oven                  30
Refrigerator                    26
Fountain                        24
Gas stove                       23
Coffeemaker                     21
Bathtub                         18
Wine rack                       17
Jacuzzi                         16
Ceiling fan                     11
Tree house                      11
Shower    

859it [00:15, 54.40it/s]


Saving labels to: ./validation/validation_labels.json...
Using train-annotations-bbox.csv for annotations...


4it [00:00, 36.02it/s]

On dataset: train
Classes we're using:
 Stairs                         5981
Couch                          4259
Swimming pool                  3881
Porch                          3854
Television                     3789
Fountain                       3691
Bed                            3563
Pillow                         3508
Countertop                     3113
Kitchen & dining room table    2127
Sink                           1648
Mirror                         1572
Sofa bed                       1501
Toilet                         1099
Billiard table                  912
Fireplace                       711
Washing machine                 655
Oven                            637
Refrigerator                    592
Bathtub                         545
Gas stove                       526
Microwave oven                  485
Ceiling fan                     478
Towel                           338
Coffeemaker                     323
Wine rack                       254
Shower                  

34835it [13:13, 43.89it/s]


Saving labels to: ./train/train_labels.json...
CPU times: user 14min, sys: 8.08 s, total: 14min 8s
Wall time: 14min 4s


In [15]:
from detectron2.data import DatasetCatalog, MetadataCatalog

def register_datasets(train, valid, target):
  for d in [train, valid]:
    dataset_name = d.split("/")[-1]
    print("Registering: {}".format(dataset_name))
    DatasetCatalog.register(dataset_name, lambda d=d: load_json_labels(d))
    MetadataCatalog.get(dataset_name).set(thing_classes=target)
  return MetadataCatalog.get(dataset_name)

In [16]:
three_class_metadata = register_datasets(train_path,valid_path,target)

Registering: train
Registering: validation


In [17]:

def get_evaluator(cfg, dataset_name, output_folder):
  if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator = COCOEvaluator(
        dataset_name=dataset_name,
        distributed=False,
        output_dir=output_folder
        cfg=cfg,
  )
  return evaluator

In [18]:
def do_test(cfg, model):
  results = OrderedDict()
  for dataset_name in cfg.DATASETS.TEST:
    data_loader = build_detection_test_loader(cfg, dataset_name)
    evaluator = get_evaluator(
        cfg, 
        dataset_name,
        output_folder=os.path.join(cfg.OUTPUT_DIR, "inference", dataset_name)
    )
    results_i = inference_on_dataset(model,
                                     data_loader,
                                     evaluator)
    results[dataset_name] = results_i
    print("### Returning results_i...")
    if comm.is_main_process():
      logger.info("Evaluation results for {} in csv format:".format(dataset_name))
      print_csv_format(results_i)

    return results

In [20]:
## mostly from detectron2 code 
def do_train(cfg, model, resume=False):

  model.train()
 
  optimizer = build_optimizer(cfg, model)

  scheduler = build_lr_scheduler(cfg, optimizer)
  print(f"Scheduler: {scheduler}")
    start_iter = (

      checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1
  )

  max_iter = cfg.SOLVER.MAX_ITER

  writers = (
      [
       CommonMetricPrinter(max_iter=max_iter),
       JSONWriter(os.path.join(cfg.OUTPUT_DIR, "metrics.json")),
       TensorboardXWriter(cfg.OUTPUT_DIR),
      ]
      if comm.is_main_process()
      else []
  )

  data_loader = build_detection_train_loader(cfg)
  logger.info("Starting training from iteration {}".format(start_iter))

  with EventStorage(start_iter) as storage:
    for data, iteration in zip(data_loader, range(start_iter, max_iter)):
      iteration = iteration + 1
      storage.step() 
      loss_dict = model(data)
      losses = sum(loss_dict.values())
      assert torch.isfinite(losses).all(), loss_dict


      loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
      losses_reduced = sum(loss for loss in loss_dict_reduced.values())
      if comm.is_main_process():
            storage.put_scalars(total_loss=losses_reduced,
                            **loss_dict_reduced)

      optimizer.zero_grad()
      losses.backward()
      optimizer.step()
      storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
      scheduler.step()


          comm.synchronize()
      periodic_checkpointer.step(iteration)

In [21]:

def setup(args):

  cfg = get_cfg()
  cfg.merge_from_file(args.config_file) 
  cfg.merge_from_list(args.opts)
  cfg.freeze()
  default_setup(
      cfg, args
  ) 
  cfg_yaml = cfg.load_yaml_with_base(os.path.join(cfg.OUTPUT_DIR, "config.yaml"))
  return cfg

In [22]:

  cfg = setup(args)
  model = build_model(cfg)
  logger.info("Model:\n{}".format(model))
  do_train(cfg, model)
  do_test(cfg, model)

In [23]:
# models_to_try = {
#     "R50-FPN-1x": "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml",
#     "R50-FPN-3x": "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
#     "R101-FPN-3x": "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml",
#     "X101-FPN-3x": "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml",
#     "RN-R50-1x": "COCO-Detection/retinanet_R_50_FPN_1x.yaml",
#     "RN-R50-3x": "COCO-Detection/retinanet_R_50_FPN_3x.yaml",
#     "RN-R101-3x": "COCO-Detection/retinanet_R_101_FPN_3x.yaml"
# }

models_to_try = {
  "RN-R101-3x": "COCO-Detection/retinanet_R_101_FPN_3x.yaml"
}

In [28]:
parser = default_argument_parser() 
weights = []
configs = []
for k, v in models_to_try.items():
  model = models_to_try[k]
  model_weights = model_zoo.get_checkpoint_url(model)
  weights.append(model_weights)
  config_file = model_zoo.get_config_file(model)
  configs.append(config_file)
weights_configs = dict(zip(weights, configs))
weights_configs

{'https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/retinanet_R_101_FPN_3x/190397697/model_final_971ab9.pkl': '/opt/conda/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml'}

In [32]:
# 测试参数
workers = 2
ims_per_batch = 2
lr = 0.00018 
max_iter = 90000 
warmup_iters = 2000 
num_classes = len(target)

arg_strings = []
for weights, config in weights_configs.items():
  arg_string = f"--config-file {config} \
                MODEL.WEIGHTS {weights} \
                DATASETS.TRAIN ('train',) \
                DATASETS.TEST ('validation',) \
                DATALOADER.NUM_WORKERS {workers} \
                SOLVER.IMS_PER_BATCH {ims_per_batch} \
                SOLVER.BASE_LR {lr} \
                SOLVER.MAX_ITER {max_iter} \
                SOLVER.WARMUP_ITERS {warmup_iters} \
                SEED {seed} \
                MODEL.ROI_HEADS.NUM_CLASSES {num_classes}".split()
  arg_strings.append(arg_string)

In [33]:

# Create different arg parsers
arg_list = []
for arg_string in arg_strings:
  args = parser.parse_args(arg_string)
  print(args)
  arg_list.append(args)

Namespace(config_file='/opt/conda/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml', dist_url='tcp://127.0.0.1:50152', eval_only=False, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', 'https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/retinanet_R_101_FPN_3x/190397697/model_final_971ab9.pkl', 'DATASETS.TRAIN', "('train',)", 'DATASETS.TEST', "('validation',)", 'DATALOADER.NUM_WORKERS', '2', 'SOLVER.IMS_PER_BATCH', '2', 'SOLVER.BASE_LR', '0.00018', 'SOLVER.MAX_ITER', '90000', 'SOLVER.WARMUP_ITERS', '2000', 'SEED', '30', 'MODEL.ROI_HEADS.NUM_CLASSES', '30'], resume=False)


In [34]:
## start to training
cfg = setup(arg_list)
model = build_model(cfg)
logger.info("Model:\n{}".format(model))
do_train(cfg, model)
do_test(cfg, model)

Running experiment: 1
WARNING [12/08 19:56:14 fvcore.common.config]: Loading config /opt/conda/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[12/08 19:56:14 detectron2]: Rank of current process: 0. World size: 1
[12/08 19:56:14 detectron2]: Environment info:
----------------------  ------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 02:25:08) [GCC 7.5.0]
numpy                   1.18.5
detectron2              0.2.1 @/opt/conda/lib/python3.7/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.0
detectron2 arch flags   sm_35, sm_37, sm_50, sm_52, sm_60, sm_61, sm_70, sm_75
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.4.0+cu100 @/home/jupyter/.local/lib/python3.7/si